In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [2]:
# @hidden_cell
CLIENT_ID = '3KER0AYTDMG0GXCPJUELSJVEK34HX54ETTPXDNT04NUZSKJY' # your Foursquare ID
CLIENT_SECRET = '0BQJPCG2R5YQ4BVMMYSKJGGQT5VMK5T5DY15HRTECD20OKOR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [3]:
address = 'Luxor, Egypt'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = 25.6872
longitude = 32.6396
radius = '10000'

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [5]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))
items = results['response']['groups'][0]['items']
items

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b73e070f964a52099be2de3',
   'name': 'Luxor Temple (معبد الأقصر)',
   'location': {'address': 'Corniche',
    'lat': 25.700380692211112,
    'lng': 32.640216096190365,
    'labeledLatLngs': [{'label': 'display',
      'lat': 25.700380692211112,
      'lng': 32.640216096190365}],
    'distance': 1468,
    'cc': 'EG',
    'city': 'الأقصر',
    'state': 'الأقصر',
    'country': 'مصر',
    'formattedAddress': ['Corniche', 'الأقصر', 'الأقصر', 'مصر']},
   'categories': [{'id': '4deefb944765f83613cdba6e',
     'name': 'Historic Site',
     'pluralName': 'Historic Sites',
     'shortName': 'Historic Site',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/historicsite_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4b73e070f964a52

In [6]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 4 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,5231e4358bbd209e540e91de,Gerda's Garden Restaurant,German,"Luxor, Egypt",28844,25.717352,32.650895
1,520a4b8793cd8c429aa8f8b7,Olives Restaurant,Mediterranean,"Luxor, Egypt",13,25.714607,32.651453
2,5288beb111d2f30d7aa27729,Hilton Luxor Spa,Spa,"Luxor, Egypt",13,25.716385,32.652612
3,50aba7cc011ca7991beb1572,Sofitel Winter Palace Luxor,Hotel,"Luxor, Egypt",11432,25.696681,32.637133


In [45]:
# create map of Helsinki using latitude and longitude values
map = folium.Map(location=[lat, lng], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)

add_markers(df)


map